In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random
from urllib.parse import unquote

# 您提供的原始 URL 列表 (保持原样，代码会自动转换为 zh-cn)
urls = [
    # 皇帝
    "https://zh.wikipedia.org/wiki/朱元璋",
    "https://zh.wikipedia.org/wiki/建文帝",
    "https://zh.wikipedia.org/wiki/明成祖",
    "https://zh.wikipedia.org/wiki/明仁宗",
    "https://zh.wikipedia.org/wiki/明宣宗",
    "https://zh.wikipedia.org/wiki/明英宗",
    "https://zh.wikipedia.org/wiki/景泰帝",
    "https://zh.wikipedia.org/wiki/明宪宗",
    "https://zh.wikipedia.org/wiki/明孝宗",
    "https://zh.wikipedia.org/wiki/明武宗",
    "https://zh.wikipedia.org/wiki/明世宗",
    "https://zh.wikipedia.org/wiki/明穆宗",
    "https://zh.wikipedia.org/wiki/明神宗",
    "https://zh.wikipedia.org/wiki/明光宗",
    "https://zh.wikipedia.org/wiki/明熹宗",
    "https://zh.wikipedia.org/wiki/崇祯帝",
    
    # 开国功臣
    "https://zh.wikipedia.org/wiki/徐达",
    "https://zh.wikipedia.org/wiki/常遇春",
    "https://zh.wikipedia.org/wiki/刘伯温",
    "https://zh.wikipedia.org/wiki/李善长",
    "https://zh.wikipedia.org/wiki/汤和",
    "https://zh.wikipedia.org/wiki/邓愈",
    "https://zh.wikipedia.org/wiki/沐英",
    "https://zh.wikipedia.org/wiki/蓝玉",
    
    # 著名文臣/内阁首辅
    "https://zh.wikipedia.org/wiki/姚广孝",
    "https://zh.wikipedia.org/wiki/于谦",
    "https://zh.wikipedia.org/wiki/王守仁",
    "https://zh.wikipedia.org/wiki/张居正",
    "https://zh.wikipedia.org/wiki/海瑞",
    "https://zh.wikipedia.org/wiki/严嵩",
    "https://zh.wikipedia.org/wiki/徐阶",
    "https://zh.wikipedia.org/wiki/高拱",
    "https://zh.wikipedia.org/wiki/杨廷和",
    "https://zh.wikipedia.org/wiki/李东阳",
    "https://zh.wikipedia.org/wiki/谢迁",
    "https://zh.wikipedia.org/wiki/刘健_(明朝)",
    "https://zh.wikipedia.org/wiki/夏言",
    "https://zh.wikipedia.org/wiki/徐光启",
    "https://zh.wikipedia.org/wiki/杨继盛",
    
    # 著名武将
    "https://zh.wikipedia.org/wiki/戚继光",
    "https://zh.wikipedia.org/wiki/俞大猷",
    "https://zh.wikipedia.org/wiki/袁崇焕",
    "https://zh.wikipedia.org/wiki/熊廷弼",
    "https://zh.wikipedia.org/wiki/孙承宗",
    "https://zh.wikipedia.org/wiki/卢象升",
    "https://zh.wikipedia.org/wiki/李成梁",
    "https://zh.wikipedia.org/wiki/李如松",
    "https://zh.wikipedia.org/wiki/祖大寿",
    "https://zh.wikipedia.org/wiki/吴三桂",
    "https://zh.wikipedia.org/wiki/史可法",
    "https://zh.wikipedia.org/wiki/秦良玉",
    
    # 宦官
    "https://zh.wikipedia.org/wiki/郑和",
    "https://zh.wikipedia.org/wiki/魏忠贤",
    "https://zh.wikipedia.org/wiki/王振_(明朝宦官)",
    "https://zh.wikipedia.org/wiki/刘瑾",
    
    # 思想家/文学家/其他
    "https://zh.wikipedia.org/wiki/唐寅",
    "https://zh.wikipedia.org/wiki/文徵明",
    "https://zh.wikipedia.org/wiki/李贽",
    "https://zh.wikipedia.org/wiki/黄宗羲",
    "https://zh.wikipedia.org/wiki/顾炎武",
    "https://zh.wikipedia.org/wiki/王夫之",
    "https://zh.wikipedia.org/wiki/李时珍",
    "https://zh.wikipedia.org/wiki/宋应星",

    # 农民起义领袖
    "https://zh.wikipedia.org/wiki/李自成",
    "https://zh.wikipedia.org/wiki/张献忠"
]

def scrape_wiki_bio_simplified(url_list, save_folder="ming_dynasty_cn"):
    # 创建保存文件夹
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
        print(f"已创建文件夹: {save_folder}")

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'zh-CN,zh;q=0.9' # 显式告诉服务器我们需要中文
    }

    print(f"开始爬取 {len(url_list)} 个条目 (简体中文版 + 参考文献)...\n")

    for raw_url in url_list:
        try:
            # 1. URL 转换：将 /wiki/ 替换为 /zh-cn/ 以获取简体版本
            target_url = raw_url.replace("/wiki/", "/zh-cn/")
            
            # 提取文件名
            name = unquote(raw_url.split("/")[-1])
            file_path = os.path.join(save_folder, f"{name}.txt")

            print(f"正在处理: {name} ...", end="", flush=True)

            response = requests.get(target_url, headers=headers, timeout=15)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                content_div = soup.find('div', {'id': 'mw-content-text'})
                
                # --- A. 提取正文 ---
                text_content = f"人物: {name}\n来源链接: {target_url}\n"
                text_content += "="*50 + "\n\n"
                
                # 提取所有段落 <p>
                paragraphs = content_div.find_all('p')
                for p in paragraphs:
                    # 移除段落中的引用标记 [1], [2] 等，让正文更干净 (可选)
                    # if p.find('sup', class_='reference'): 
                    #     for tag in p.find_all('sup', class_='reference'): tag.decompose()
                    
                    text = p.get_text().strip()
                    if text:
                        text_content += text + "\n\n"

                # --- B. 提取参考文献 ---
                text_content += "\n" + "="*20 + " 参考文献 " + "="*20 + "\n\n"
                
                # 维基百科参考文献通常位于 class="references" 的有序列表 <ol> 中
                ref_lists = soup.find_all('ol', class_='references')
                
                ref_count = 1
                if ref_lists:
                    for ref_ol in ref_lists:
                        # 遍历每一个引用项 <li>
                        for li in ref_ol.find_all('li'):
                            # 移除其中的 "跳回" 链接 (通常是 ^ 符号)
                            for jump_link in li.find_all('a', href=True):
                                if "#cite_ref" in jump_link['href']:
                                    jump_link.decompose() # 删除跳转链接
                                    
                            ref_text = li.get_text().strip()
                            if ref_text:
                                text_content += f"[{ref_count}] {ref_text}\n"
                                ref_count += 1
                else:
                    text_content += "（未检测到参考文献列表）\n"

                # --- 保存文件 ---
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(text_content)
                
                print(" [成功]")
            else:
                print(f" [失败: 状态码 {response.status_code}]")

            time.sleep(random.uniform(1.5, 3.5)) # 稍长的延迟

        except Exception as e:
            print(f"\n处理 {raw_url} 时发生错误: {e}")

    print("\n所有任务完成！")

if __name__ == "__main__":
    scrape_wiki_bio_simplified(urls)

开始爬取 36 个条目 (简体中文版 + 参考文献)...

正在处理: 朱元璋 ... [成功]
正在处理: 建文帝 ... [成功]
正在处理: 明成祖 ... [成功]
正在处理: 明仁宗 ... [成功]
正在处理: 明宣宗 ... [成功]
正在处理: 明英宗 ... [成功]
正在处理: 景泰帝 ... [成功]
正在处理: 明宪宗 ... [成功]
正在处理: 明孝宗 ... [成功]
正在处理: 明武宗 ... [成功]
正在处理: 明世宗 ... [成功]
正在处理: 明穆宗 ... [成功]
正在处理: 明神宗 ... [成功]
正在处理: 明光宗 ... [成功]
正在处理: 明熹宗 ... [成功]
正在处理: 崇祯帝 ... [成功]
正在处理: 徐达 ... [成功]
正在处理: 常遇春 ... [成功]
正在处理: 刘伯温 ... [成功]
正在处理: 李善长 ... [成功]
正在处理: 汤和 ... [成功]
正在处理: 邓愈 ... [成功]
正在处理: 沐英 ... [成功]
正在处理: 蓝玉 ... [成功]
正在处理: 姚广孝 ... [成功]
正在处理: 郑和 ... [成功]
正在处理: 于谦 ... [成功]
正在处理: 王守仁 ... [成功]
正在处理: 张居正 ... [成功]
正在处理: 海瑞 ... [成功]
正在处理: 戚继光 ... [成功]
正在处理: 俞大猷 ... [成功]
正在处理: 袁崇焕 ... [成功]
正在处理: 熊廷弼 ... [成功]
正在处理: 孙承宗 ... [成功]
正在处理: 卢象升 ... [成功]

所有任务完成！
